# 물체 검출 결과 평가 방법 이해하기
성능 평가 코드를 이해하고 분석하여 precision-recall curve에서 주어진 recall값에 대한 threshold 값을 계산합니다.<br/>
구해진 threshold값을 이용하여 검출 결과를 출력합니다.

In [4]:
"""Adapted from:
    @longcw faster_rcnn_pytorch: https://github.com/longcw/faster_rcnn_pytorch
    @rbgirshick py-faster-rcnn https://github.com/rbgirshick/py-faster-rcnn
    Licensed under The MIT License [see LICENSE for details]
"""

from __future__ import print_function
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from data import VOC_ROOT, VOCAnnotationTransform, VOCDetection, BaseTransform
import torch.utils.data as data
from utils import Timer
from ssd import build_ssd

import sys
import os
import time
import argparse
import numpy as np
import pickle
import cv2

# %matplotlib inline
from matplotlib import pyplot as plt

import xml.etree.ElementTree as ET

labelmap = (  # always index 0
    'aeroplane', 'bicycle', 'bird', 'boat',
    'bottle', 'bus', 'car', 'cat', 'chair',
    'cow', 'diningtable', 'dog', 'horse',
    'motorbike', 'person', 'pottedplant',
    'sheep', 'sofa', 'train', 'tvmonitor')

annopath = os.path.join(VOC_ROOT, 'VOC2007', 'Annotations', '%s.xml')
imgpath = os.path.join(VOC_ROOT, 'VOC2007', 'JPEGImages', '%s.jpg')
imgsetpath = os.path.join(VOC_ROOT, 'VOC2007', 'ImageSets', 'Main', '{:s}.txt')
YEAR = '2007'
devkit_path = VOC_ROOT + 'VOC' + YEAR
dataset_mean = (104, 117, 123)

if torch.cuda.is_available():    
    torch.set_default_tensor_type('torch.cuda.FloatTensor')    
else:
    torch.set_default_tensor_type('torch.FloatTensor')    

In [5]:
### load net
num_classes = len(labelmap) + 1                      # +1 for background
net = build_ssd('vgg16', 300, 21)    # initialize SSD
# net.load_state_dict(torch.load(args.trained_model))
net.load_weights('weights/ssd300_mAP_77.43_v2.pth')
# net.load_state_dict(torch.load(args.trained_model, map_location='cpu'))
net.eval()
print('Finished loading model!')

### load data
set_type = 'test'
dataset = VOCDetection(VOC_ROOT, [('2007', set_type)],
                       BaseTransform(300, dataset_mean),
                       VOCAnnotationTransform())
if torch.cuda.is_available():
    net = net.cuda()
    cudnn.benchmark = True

Loading weights into state dict...
Finished!
Finished loading model!


In [6]:
save_folder = 'ssd300_results'
top_k = 10

from layers import Detect

num_images = len(dataset)
parser = Detect(num_classes, 0, 200, 0.01, 0.45)
softmax = nn.Softmax(dim=-1)

# all detections are collected into:
#    all_boxes[cls] = N x 6 array of detections in
#    (x1, y1, x2, y2, score, ids)
all_boxes =[np.zeros((0,6)) for _ in range(len(labelmap)+1)]

# timers
_t = {'forward': Timer(), 'misc': Timer()}    
        
output_dir = os.path.join(save_folder, set_type)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
det_file = os.path.join(output_dir, 'detections.pkl')

if not os.path.exists(det_file):
    for i in range(num_images):    
        im, gt, h, w, ids = dataset.pull_item_with_index(i)

        x = im.unsqueeze(0)
        if torch.cuda.is_available():
            x = x.cuda()

        _t['forward'].tic()
        with torch.no_grad():    
            loc_pred, cls_pred, priors = net(x)            
        forward_time = _t['forward'].toc(average=True)    

        _t['misc'].tic()
        detections = parser(loc_pred, softmax(cls_pred), priors.type(type(x)))                
        # skip j = 0, because it's the background class
        for j in range(1, detections.size(1)):
            dets = detections[0, j, :]
            mask = dets[:, 0].gt(0.).expand(5, dets.size(0)).t()
            dets = torch.masked_select(dets, mask).view(-1, 5)
            if dets.shape[0] == 0:
                continue
            boxes = dets[:, 1:]
            boxes[:, 0] *= w
            boxes[:, 2] *= w
            boxes[:, 1] *= h
            boxes[:, 3] *= h
            scores = dets[:, 0].cpu().numpy()

            ids_dets = ids*np.ones( (len(scores),1) )
            cls_dets = np.hstack(
                (boxes.cpu().numpy(),\
                 scores[:, np.newaxis], \
                 ids_dets
                )).astype(np.float32, copy=False)                
            all_boxes[j] = np.vstack( (all_boxes[j], cls_dets) )
            
        misc_time = _t['misc'].toc(average=True)
        
        if i % 100 == 0:
            print('[im_detect: {:d}/{:d}] forward: {:.3f}s, misc: {:.3f}s'.format(i+1, num_images, forward_time, misc_time))
            
#     for j in range(num_images):
#         all_boxes[j] = np.array(all_boxes[j])

    with open(det_file, 'wb') as f:
        pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)    
        
    print('Saved to {:}.'.format(det_file))
else:
    with open(det_file, 'rb') as f:
        all_boxes = pickle.load(f)    
    print('Load from {:}.'.format(det_file))
        


Load from ssd300_results/test/detections.pkl.


In [7]:
def parse_rec(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['pose'] = obj.find('pose').text
        obj_struct['truncated'] = int(obj.find('truncated').text)
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text) - 1,
                              int(bbox.find('ymin').text) - 1,
                              int(bbox.find('xmax').text) - 1,
                              int(bbox.find('ymax').text) - 1]
        objects.append(obj_struct)

    return objects

In [19]:
def voc_eval(det_boxes, annopath, imagesetfile, classname, cachedir,
             ovthresh=0.5, use_07_metric=True):
    """rec, prec, ap = voc_eval(detpath,
                           annopath,
                           imagesetfile,
                           classname,
                           [ovthresh],
                           [use_07_metric])
Top level function that does the PASCAL VOC evaluation.
detpath: Path to detections
   detpath.format(classname) should produce the detection results file.
annopath: Path to annotations
   annopath.format(imagename) should be the xml annotations file.
imagesetfile: Text file containing the list of images, one image per line.
classname: Category name (duh)
cachedir: Directory for caching the annotations
[ovthresh]: Overlap threshold (default = 0.5)
[use_07_metric]: Whether to use VOC07's 11 point AP computation
   (default True)
"""
# assumes detections are in detpath.format(classname)
# assumes annotations are in annopath.format(imagename)
# assumes imagesetfile is a text file with each line an image name
# cachedir caches the annotations in a pickle file
# first load gt
    if not os.path.isdir(cachedir):
        os.mkdir(cachedir)
    cachefile = os.path.join(cachedir, 'annots.pkl')
    # read list of images
    with open(imagesetfile, 'r') as f:
        lines = f.readlines()
    imagenames = [x.strip() for x in lines]
    if not os.path.isfile(cachefile):
        # load annots
        recs = {}
        for i, imagename in enumerate(imagenames):
            recs[imagename] = parse_rec(annopath % (imagename))
            if i % 100 == 0:
                print('Reading annotation for {:d}/{:d}'.format(
                   i + 1, len(imagenames)))
        # save
        print('Saving cached annotations to {:s}'.format(cachefile))
        with open(cachefile, 'wb') as f:
            pickle.dump(recs, f)
    else:
        # load
        with open(cachefile, 'rb') as f:
            recs = pickle.load(f)

    # extract gt objects for this class
    class_recs = {}
    npos = 0
    for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'] == classname]
        bbox = np.array([x['bbox'] for x in R])
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
        det = [False] * len(R)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox,
                                 'difficult': difficult,
                                 'det': det}

    import pdb
    pdb.set_trace()
    
    # read dets
#     detfile = detpath.format(classname)
#     with open(detfile, 'r') as f:
#         lines = f.readlines()

    if len(det_boxes):
        
#         import pdb
#         pdb.set_trace()
    
        BB = det_boxes[:,:4]
        confidence = det_boxes[:,4]
        image_ids = det_boxes[:,5]        

        # sort by confidence
        sorted_ind = np.argsort(-confidence)
        sorted_scores = np.sort(-confidence)
        BB = BB[sorted_ind, :]
        image_ids = [image_ids[x] for x in sorted_ind]

        # go down dets and mark TPs and FPs
        nd = len(image_ids)
        tp = np.zeros(nd)
        fp = np.zeros(nd)
        for d in range(nd):
            R = class_recs['%06d'%int(image_ids[d])]
            bb = BB[d, :].astype(float)
            ovmax = -np.inf
            BBGT = R['bbox'].astype(float)
            if BBGT.size > 0:
                # compute overlaps
                # intersection
                ixmin = np.maximum(BBGT[:, 0], bb[0])
                iymin = np.maximum(BBGT[:, 1], bb[1])
                ixmax = np.minimum(BBGT[:, 2], bb[2])
                iymax = np.minimum(BBGT[:, 3], bb[3])
                iw = np.maximum(ixmax - ixmin, 0.)
                ih = np.maximum(iymax - iymin, 0.)
                inters = iw * ih
                uni = ((bb[2] - bb[0]) * (bb[3] - bb[1]) +
                       (BBGT[:, 2] - BBGT[:, 0]) *
                       (BBGT[:, 3] - BBGT[:, 1]) - inters)
                overlaps = inters / uni
                ovmax = np.max(overlaps)
                jmax = np.argmax(overlaps)

            if ovmax > ovthresh:
                if not R['difficult'][jmax]:   # if matched GT is difficult, detected box will be ignored.
                    if not R['det'][jmax]:     # check duplicated detection.
                        tp[d] = 1.
                        R['det'][jmax] = 1
                    else:
                        fp[d] = 1.             # if so, detected box will be considered as false positive.
            else:
                fp[d] = 1.                     # if the detected box is not matched to any GT boxes, false positive.

        # compute precision recall
        fp = np.cumsum(fp)
        tp = np.cumsum(tp)
        rec = tp / float(npos)
        # avoid divide by zero in case the first detection matches a difficult
        # ground truth
        prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
#         ap = voc_ap(rec, prec, use_07_metric)

        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):    # recall thresholds
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        rec = -1.
        prec = -1.
        ap = -1.

    return rec, prec, ap


In [40]:
rec, prec, ap = voc_eval(all_boxes[1], annopath, imgsetpath.format(set_type), 'bicycle', cachedir, ovthresh=0.5)

> <ipython-input-19-9ad02a5f6b14>(71)voc_eval()
-> if len(det_boxes):


(Pdb)  det_boxes


array([[ 2.03795624e+00,  1.30934235e+02,  5.86419678e+01,
         2.04625092e+02,  3.10207773e-02,  2.00000000e+00],
       [ 9.75566559e+01,  6.27198982e+01,  1.38641479e+02,
         9.15373993e+01,  2.81747188e-02,  2.00000000e+00],
       [ 1.13417458e+02,  5.86000557e+01,  1.48989929e+02,
         9.11373520e+01,  2.20974404e-02,  2.00000000e+00],
       ...,
       [ 1.88180176e+02,  6.55056229e+01,  5.07051117e+02,
         2.72068268e+02,  1.02572581e-02,  9.93100000e+03],
       [ 1.34747192e+02, -2.64772224e+00,  1.99617218e+02,
         3.81005936e+01,  1.06335301e-02,  9.94800000e+03],
       [ 1.75539627e+02, -1.44076796e+01,  3.64772430e+02,
         1.39319672e+02,  3.85760851e-02,  9.96300000e+03]])


(Pdb)  c


In [38]:
print('Evaluating detections')

cachedir = os.path.join(save_folder, 'annotations_cache')
aps = []
print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))

for i, cls in enumerate(labelmap[1]):
    rec, prec, ap = voc_eval(all_boxes[i], annopath, imgsetpath.format(set_type), cls, cachedir, ovthresh=0.5)
    aps += [ap]
    print('AP for {} = {:.4f}'.format(cls, ap))
    with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
        pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
print('Mean AP = {:.4f}'.format(np.mean(aps)))
print('~~~~~~~~')
print('Results:')
for ap in aps:
    print('{:.3f}'.format(ap))
print('{:.3f}'.format(np.mean(aps)))
print('~~~~~~~~')
print('')
print('--------------------------------------------------------------')
print('Results computed with the **unofficial** Python eval code.')
print('Results should be very close to the official MATLAB eval code.')
print('--------------------------------------------------------------')

Evaluating detections
VOC07 metric? Yes
> <ipython-input-19-9ad02a5f6b14>(71)voc_eval()
-> if len(det_boxes):


(Pdb)  det_boxes


array([], shape=(0, 6), dtype=float64)


(Pdb)  c


AP for b = -1.0000
> <ipython-input-19-9ad02a5f6b14>(71)voc_eval()
-> if len(det_boxes):


(Pdb)  c


/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in true_divide
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in greater_equal


AP for i = 0.0000
> <ipython-input-19-9ad02a5f6b14>(71)voc_eval()
-> if len(det_boxes):


(Pdb)  exit


BdbQuit: 

In [37]:
all_boxes[i][81,]

array([ 13.68173981,  70.3891449 , 407.73852539, 324.32394409,
         0.99882454, 261.        ])